Use the "base" environment in conda on Pop!OS.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import sklearn
import pandas as pd
#import surprise as sur
#from surprise import SVD
from surprise import Dataset, Reader
#from surprise.model_selection import train_test_split
#from surprise.model_selection import cross_validate
#from sklearn.preprocessing import OneHotEncoder
import numpy as np
import fm_utils as fm

In [3]:
df_attr = pd.read_csv("activity_reduced_with_attributes.csv")

In [4]:
df_attr['year'] = df_attr.booking_date.str[0:4]
years = [2000+d for d in range(15,23)]
df_years = {}
for year in years: 
    df_years[year] = df_attr[df_attr['year'] == str(year)]
    df_years[year].to_csv("attrib_"+str(year)+".csv", index=0)
# year is an integer

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [102]:
def create_attributes_by_year(year):
    df1 = pd.read_csv("attrib_"+str(year)+".csv")
    
    # Break up passenger age into four groups
    df1['age_departure'] = pd.cut(df1.age_at_flight, bins=[0, 30, 50, 70, 120])

    df2 = pd.get_dummies(df1, prefix=['fl_dwk', 'fl_mo', 'bk_dwk','bk_mo','age','addr'], 
                         columns=['flight_dowk','flight_mo', 'booking_dowk', 'booking_mo', 'age_departure', 'ADDR_COUNTRY'])
    df3 = df2[['MEMBER_ID','D']].copy()
    df4 = df2.loc[:, 'fl_dwk_0':'addr_Venezuela']

    df3['ratings'] = 1  # not needed?
    #df3.columns = ['userID', 'productID', 'ratings']

#     df_memb_dest = df3loc[:, 'MEMBER_ID':'D']
    df_memb_dest = df3[['MEMBER_ID', 'D']].copy()
    df_attrib = df2.loc[:, 'fl_dwk_0':'addr_Venezuela']
    df_attrib['MEMBER_ID'] = df_memb_dest.loc[:,'MEMBER_ID']
    df_memb_dest['ratings'] = 1
    
    df_memb_dest.to_csv(f"member_dest_{year}.csv", index=0)
    df_attrib.to_csv(f"member_attrib_{year}.csv", index=0)
    
    return df_memb_dest, df_attrib

In [101]:
df_members, df_attrib = create_attributes_by_year(2016)
df_members, df_attrib = create_attributes_by_year(2017)

In [68]:
df_members.columns, df_attrib.columns

(Index(['MEMBER_ID', 'TRUE_OD', 'D', 'ratings'], dtype='object'),
 Index(['fl_dwk_0', 'fl_dwk_1', 'fl_dwk_2', 'fl_dwk_3', 'fl_dwk_4', 'fl_dwk_5',
        'fl_dwk_6', 'fl_mo_1', 'fl_mo_2', 'fl_mo_3', 'fl_mo_4', 'fl_mo_5',
        'fl_mo_6', 'fl_mo_7', 'fl_mo_8', 'fl_mo_9', 'fl_mo_10', 'fl_mo_11',
        'fl_mo_12', 'bk_dwk_0', 'bk_dwk_1', 'bk_dwk_2', 'bk_dwk_3', 'bk_dwk_4',
        'bk_dwk_5', 'bk_dwk_6', 'bk_mo_1', 'bk_mo_2', 'bk_mo_3', 'bk_mo_4',
        'bk_mo_5', 'bk_mo_6', 'bk_mo_7', 'bk_mo_8', 'bk_mo_9', 'bk_mo_10',
        'bk_mo_11', 'bk_mo_12', 'age_(0, 30]', 'age_(30, 50]', 'age_(50, 70]',
        'age_(70, 120]', 'addr_Argentina', 'addr_Aruba', 'addr_Belize',
        'addr_Bolivia', 'addr_Brazil', 'addr_Canada', 'addr_Chile',
        'addr_Colombia', 'addr_Costa Rica', 'addr_Cuba',
        'addr_Dominican Republic', 'addr_Ecuador', 'addr_El Salvador',
        'addr_Guatemala', 'addr_Guyana', 'addr_Haiti', 'addr_Honduras',
        'addr_Jamaica', 'addr_Mexico', 'addr_Nicaragu

In [5]:
year = 2016
df1 = pd.read_csv("attrib_"+str(year)+".csv")
df1.shape, df1.columns

((121806, 23),
 Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
        'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
        'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
        'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
        'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'age_at_flight', 'year'],
       dtype='object'))

## construct one-hot encoded vector
Transform booking_dowk (7), booking_mo (12), flight_dowk (7), flight_mo (12), age (4)

Ages: 0-30, 30-50, 50-70, 70+ (4 categories)

In [8]:
df1['age_departure'] = pd.cut(df1.age_at_flight, bins=[0, 30, 50, 70, 120])
df1['age_departure'].value_counts()

(30, 50]     77837
(50, 70]     29870
(0, 30]      13146
(70, 120]      953
Name: age_departure, dtype: int64

In [9]:
df2 = pd.get_dummies(df1, prefix=['fl_dwk', 'fl_mo', 'bk_dwk','bk_mo','age','addr'], 
                     columns=['flight_dowk','flight_mo', 'booking_dowk', 'booking_mo', 'age_departure', 'ADDR_COUNTRY'])
df2.columns, df2.shape

(Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
        'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY', 'PNR',
        'PARTY_SZ', 'size', 'booking_date', 'flight_date', 'TIER_LEVEL',
        'GENDER', 'BIRTH_DATE', 'age_at_flight', 'year', 'fl_dwk_0', 'fl_dwk_1',
        'fl_dwk_2', 'fl_dwk_3', 'fl_dwk_4', 'fl_dwk_5', 'fl_dwk_6', 'fl_mo_1',
        'fl_mo_2', 'fl_mo_3', 'fl_mo_4', 'fl_mo_5', 'fl_mo_6', 'fl_mo_7',
        'fl_mo_8', 'fl_mo_9', 'fl_mo_10', 'fl_mo_11', 'fl_mo_12', 'bk_dwk_0',
        'bk_dwk_1', 'bk_dwk_2', 'bk_dwk_3', 'bk_dwk_4', 'bk_dwk_5', 'bk_dwk_6',
        'bk_mo_1', 'bk_mo_2', 'bk_mo_3', 'bk_mo_4', 'bk_mo_5', 'bk_mo_6',
        'bk_mo_7', 'bk_mo_8', 'bk_mo_9', 'bk_mo_10', 'bk_mo_11', 'bk_mo_12',
        'age_(0, 30]', 'age_(30, 50]', 'age_(50, 70]', 'age_(70, 120]',
        'addr_Argentina', 'addr_Aruba', 'addr_Belize', 'addr_Bolivia',
        'addr_Brazil', 'addr_Canada', 'addr_Chile', 'addr_Colombia',
        'addr_Costa Rica', '

In [52]:
df_memb_dest = df2.loc[:, 'MEMBER_ID':'D']
df_attrib = df2.loc[:, 'fl_dwk_0':'addr_Venezuela']
df_attrib['MEMBER_ID'] = df_memb_dest.loc[:,'MEMBER_ID']
df_memb_dest['ratings'] = 1
#df3.columns = ['userID', 'productID', 'ratings']
# df_memb_dest.head()

,MEMBER_ID,TRUE_OD,D,ratings
0,100031203,PTY-LIR,LIR,1
1,230019464,PTY-BOG,BOG,1
2,230019464,PTY-EZE,EZE,1
3,230019464,PTY-BOG,BOG,1
4,230019464,PTY-SJU,SJU,1


In [15]:
df4.col

Index(['fl_dwk_0', 'fl_dwk_1', 'fl_dwk_2', 'fl_dwk_3', 'fl_dwk_4', 'fl_dwk_5',
       'fl_dwk_6', 'fl_mo_1', 'fl_mo_2', 'fl_mo_3', 'fl_mo_4', 'fl_mo_5',
       'fl_mo_6', 'fl_mo_7', 'fl_mo_8', 'fl_mo_9', 'fl_mo_10', 'fl_mo_11',
       'fl_mo_12', 'bk_dwk_0', 'bk_dwk_1', 'bk_dwk_2', 'bk_dwk_3', 'bk_dwk_4',
       'bk_dwk_5', 'bk_dwk_6', 'bk_mo_1', 'bk_mo_2', 'bk_mo_3', 'bk_mo_4',
       'bk_mo_5', 'bk_mo_6', 'bk_mo_7', 'bk_mo_8', 'bk_mo_9', 'bk_mo_10',
       'bk_mo_11', 'bk_mo_12', 'age_(0, 30]', 'age_(30, 50]', 'age_(50, 70]',
       'age_(70, 120]', 'addr_Argentina', 'addr_Aruba', 'addr_Belize',
       'addr_Bolivia', 'addr_Brazil', 'addr_Canada', 'addr_Chile',
       'addr_Colombia', 'addr_Costa Rica', 'addr_Cuba',
       'addr_Dominican Republic', 'addr_Ecuador', 'addr_El Salvador',
       'addr_Guatemala', 'addr_Guyana', 'addr_Haiti', 'addr_Honduras',
       'addr_Jamaica', 'addr_Mexico', 'addr_Nicaragua', 'addr_Panama',
       'addr_Paraguay', 'addr_Peru', 'addr_Puerto Rico',
   

In [12]:
reader = Reader(rating_scale=[1,1])
data = Dataset.load_from_df(df3, reader)

In [13]:
reader = Reader(rating_scale=[1, 1])   # All ratings are 1
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
data_training = data.build_full_trainset()
# numbers start from 1 in columns

NameError: name 'df' is not defined

In [8]:
reader = Reader(rating_scale=[1, 1])   # All ratings are 1
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
data_training = data.build_full_trainset()
# numbers start from 1 in columns

In [47]:
data_training.ur[101]

[(39, 1.0), (40, 1.0)]

In [48]:
dat = fm.create_fn_text_data(data_training, df)

(62991, 1) (62991, 1) (62991, 1)


In [11]:
text = []
for i in range(dat.shape[0]):
    line = " ".join(dat[i])
    text.append(line)
full_text = "\n".join(text)
# print(full_text)
text_file = open("fm_input.txt", "w")
text_file.write(full_text)
text_file.close()

In [12]:
dat

array([['1.0', '0:1', '23562:1'],
       ['1.0', '1:1', '23563:1'],
       ['1.0', '2:1', '23564:1'],
       ...,
       ['1.0', '23560:1', '23593:1'],
       ['1.0', '23560:1', '23566:1'],
       ['1.0', '23561:1', '23574:1']], dtype='<U32')

In [13]:
df.columns

Index(['userID', 'itemID', 'rating'], dtype='object')